In [167]:
import math
import string
import torch
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [168]:
names =[]
with open("../code/names.txt") as file:
    names = file.read().split("\n")
stoi = {c:i for i, c in enumerate(string.ascii_lowercase)}
itos = {c:i for i, c in stoi.items()}

stoi["."] = 26
itos[26] = "."

BLOCK_SIZE = 3
inputs = []
labels = []


for name in names:
    context = [26] * BLOCK_SIZE
    for c in name + ".":
        label = stoi[c]
        inputs.append(context)
        labels.append(label)
#         print("".join(itos[i] for i in context), '-->', itos[label])
        context = context[1:] + [label]

In [169]:
inputs = torch.tensor(inputs)
labels = torch.tensor(labels)

In [170]:
# each character is a 5D vector, randomly initialize char embedding
CHAR_EMBEDDINGS = torch.randn((27,5)) 
WORD_EMBEDDINGS = CHAR_EMBEDDINGS[inputs]

# rand initialize weights and biases for Layer 1 
W1 = torch.randn((15, 100))
b1 = torch.randn(100)

print(CHAR_EMBEDDINGS.shape)
print(WORD_EMBEDDINGS.shape)
print(W1.shape)
print(b1.shape)

torch.Size([27, 5])
torch.Size([228147, 3, 5])
torch.Size([15, 100])
torch.Size([100])


In [171]:
# # Pass it through layer 1 with 100 neurons
# h = torch.tanh(WORD_EMBEDDINGS.view(-1,15) @ W1 + b1)

# # rand initialize weights and biases for Layer 2
# W2 = torch.randn((100, 27))
# b2 = torch.randn(27)

# # Pass through Layer 2 (final layer)

# logits = h @ W2 + b2
# logits = logits.exp() # Logits -> log counts -> Softmax to mimic probability distribution as output
# logits = logits / logits.sum(dim=1, keepdim=True)
# loss = -logits[torch.arange(len(inputs)), labels].log().mean()

In [172]:
# # Cross Entropy
# logits = h @ W2 + b2
# loss = F.cross_entropy(logits, labels)

In [173]:
parameters = [CHAR_EMBEDDINGS, W1, W2, b1, b2]

In [174]:
# EPOCHS = 25
# LEARNING_RATE = 1

# for epoch in range(EPOCHS):
    
#     # Require grad
#     for p in parameters:
#         p.requires_grad = True
    
#     # Forward pass
#     WORD_EMBEDDINGS = CHAR_EMBEDDINGS[inputs]
#     h = torch.tanh(WORD_EMBEDDINGS.view(-1,15) @ W1 + b1)
#     logits = h @ W2 + b2
    
#     # Loss
#     loss = F.cross_entropy(logits, labels)
    
#     for p in parameters:
#         p.grad = None
        
#     # Backward pass
#     loss.backward()
    
#     for p in parameters:
#         p.data += -LEARNING_RATE * p.grad
#     print(epoch, loss)

In [175]:
# # We can use mini-batches too

# EPOCHS = 25
# LEARNING_RATE = 0.1
# BATCH_SIZE = 10

# for epoch in range(EPOCHS):
    
#     # Require grad
#     for p in parameters:
#         p.requires_grad = True
        
#     batch_idx = torch.randint(0, inputs.shape[0], (BATCH_SIZE,))
    
#     # Forward pass
#     WORD_EMBEDDINGS = CHAR_EMBEDDINGS[inputs[batch_idx]]
#     h = torch.tanh(WORD_EMBEDDINGS.view(-1,15) @ W1 + b1)
#     logits = h @ W2 + b2
    
#     # Loss
#     loss = F.cross_entropy(logits, labels[batch_idx])
    
#     for p in parameters:
#         p.grad = None
        
#     # Backward pass
#     loss.backward()
    
#     for p in parameters:
#         p.data += -LEARNING_RATE * p.grad
#     print(epoch, loss)

In [177]:
# Use Dynamic Learning Rates

lre = torch.linspace(-3, 0, 30000)
lrs = [10**lr for lr in lre]
lr_exps = []
lr_used = []

losses = []

# We can use mini-batches too

EPOCHS = 30000
BATCH_SIZE = 10

for epoch in range(EPOCHS):
    
    # Require grad
    for p in parameters:
        p.requires_grad = True
        
    batch_idx = torch.randint(0, inputs.shape[0], (BATCH_SIZE,))
    
    # Forward pass
    WORD_EMBEDDINGS = CHAR_EMBEDDINGS[inputs[batch_idx]]
    h = torch.tanh(WORD_EMBEDDINGS.view(-1,15) @ W1 + b1)
    logits = h @ W2 + b2
    
    # Loss
    loss = F.cross_entropy(logits, labels[batch_idx])
    
    for p in parameters:
        p.grad = None
    
    # Backward pass
    loss.backward()
    
    # Update
    lr = lrs[epoch]
    for p in parameters:
        p.data += -lr * p.grad
    
    lr_exps.append(lre[epoch])
    lr_used.append(lr)
    losses.append(loss.item())

In [178]:
loss

tensor(2.6018, grad_fn=<NllLossBackward0>)

In [ ]:
# plt.plot(lr_idx, losses)

In [ ]:
# plt.plot(lr_used, losses)

In [179]:
# Infer from model

g = torch.Generator().manual_seed(124578798232)
idx = 0
BLOCK_SIZE = 3
NO_OF_WORDS = 10
words = []

for i in range(NO_OF_WORDS):
    word = ""
    context = [26] * BLOCK_SIZE
    
    while True:
        WORD_EMBEDDINGS = CHAR_EMBEDDINGS[torch.tensor([context])]
        h = torch.tanh(WORD_EMBEDDINGS.view(1,-1) @ W1 + b1)
        logits = h @ W2 + b2
        logits = F.softmax(logits, dim=1)
        pred_idx = torch.multinomial(logits, num_samples=1, replacement=True, generator=g).item()
        context = context[1:] + [pred_idx]
        word += itos[pred_idx]
        
        if pred_idx == 26:
            words.append(word)
            break
words

['nihn.',
 'lin.',
 'zun.',
 'el.',
 'raere.',
 'nan.',
 'dasnoa.',
 'ambryn.',
 'kinn.',
 'rua.']